In [14]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 

samples=os.listdir("SR_OS")
print(samples)
if "MJ.root" in samples:
    os.system("rm SR_OS/MJ.root")

def biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def normalization(hist_list,norm_bin):
    for hist in hist_list:
        for i in range(1,hist.GetNbinsX()+2):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/norm_bin
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)

['Z_centrality.pdf', 'pt_bal.pdf', 'ljet0_pt.pdf', 'rnn_score_3p.pdf', 'rnn_score_1p.pdf', 'delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt.pdf', 'tau_pt.pdf', 'tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt.pdf', 'Ztautau_Sherpa.root', 'Wjets.root', 'n_bjets.pdf', 'delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt.pdf', 'reco_mass_o.pdf', 'singletop.root', 'n_jets_interval.pdf', 'Zjets.root', 'lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt.pdf', 'eBDT.pdf', 'Signal_Sherpa.root', 'delta_phi.pdf', 'mass_jj.pdf', 'delta_y.pdf', 'delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt.pdf', 'lep_pt.pdf', 'VV.root', 'MJ.root', 'leptau_mass.pdf', 'Data.root', 'omega.pdf', 'ttbar.root', 'BG.root', 'lepiso.pdf', 'reco_mass_i.pdf', 'ljet1_pt.pdf', 'M

In [15]:
histos={
"delta_phi":["n",1,1,1,"Events/0.1","#Delta#varphi(#tau,#mu)",1.4,2.3],
"n_bjets":["n",1,1,1,"Events/1","N b-jets",1.4,5],
"lepiso":["n",1,1,1,"Events/1","Lep Iso",1.4,2],
"n_jets_interval":["n",1,1,1,"Events/1","N gap jets",1.4,2],
"lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt":["yy",0.1,0.2,0.2,"Events/0.2","#eta(#mu)",1.2,2.5],
"tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt":["yy",0.1,0.2,0.2,"Events/0.2","#eta(#tau)",1.2,2.5],
"delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt":["yy",0.2,0.2,0.2,"Events/0.2","#DeltaR(#tau,#mu)",1.2,2],
"delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt":["yy",0.2,0.2,0.2,"Events/0.2","#DeltaR(#mu,j)",1.2,2],
"delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt":["yy",0.2,0.2,0.2,"Events/0.2","#DeltaR(#tau,j)",1.2,2],

"delta_y":["yy",0.8,0.4,0.4,"Events/0.4","#Delta#eta(j1,j2)",1.2,4,1.8,10],
"omega":["yy",1,0.2,0.2,"Events/0.2","#Omega",1.2,3,-0.2,1.6],
"rnn_score_1p":["yy",0.2,0.05,0.05,"Events/0.05","jetRNNScore1p",1.2,1,0.25,1],
"rnn_score_3p":["yy",0.8,0.1,0.1,"Events/0.1","jetRNNScore1p",1.2,1,0.4,1],
"lep_pt":["yy",120,10,20,"Events/10 GeV","#mu(p_T)",1.2,200,25,200],
"tau_pt":["yy",120,10,20,"Events/10 GeV","#tau(p_T)",1.2,200,25,200],
"ljet0_pt":["yy",120,10,20,"Events/10 GeV","j1(p_T)",1.2,200,55,200],
"ljet1_pt":["yy",120,10,20,"Events/10 GeV","j2(p_T)",1.2,200,45,200],
"pt_bal":["yy",0.4,0.04,0.05,"Events/0.04","p_T balance",1.2,1,0.15,0],
"Z_centrality":["yy",1,0.1,0.5,"Events/0.1","Z centrality",1.2,5,0.5,0],
"reco_mass_i":["yy",120,5,20,"Events/5 GeV","m_{reco}(GeV)(i)",1.2,240,65,115],
"reco_mass_o":["yy",120,5,20,"Events/5 GeV","m_{reco}(GeV)(o)",1.2,240,65,115],
"mass_jj":["yy",1500,250,500,"Events/250 GeV","m(j1,j2)",1.2,3000,250,3000],
"eBDT":["yy",0.05,0.01,0.05,"Events/0.01","eBDTScore",30,1,1,0.05],
"leptau_mass":["yy",120,5,20,"Events/5 GeV","m(e,#tau)(GeV)",1.2,200,80,0],
}

In [16]:
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
signal_sample="Signal_Sherpa.root"
ztt_sample="Ztautau_Sherpa.root"
production=True

#### CREATING THE CORRESPONDING MC.ROOT FILE EVERYTIME YOU CHANGE THE SIGNAL SAMPLE

for i in ["SR_OS/","SR_SS/","CR_OS/","CR_SS/"]:
    os.system("rm "+i+"MC.root")
    os.chdir(i)
    os.system("hadd MC.root "+"BG.root "+signal_sample)
    os.chdir("..")

signal_sample_file=ROOT.TFile.Open("SR_OS/"+signal_sample,"READ")

tau_matched_cuts_1p=signal_sample_file.Get("tau_matched_1p_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco")
tau_matched_cuts_3p=signal_sample_file.Get("tau_matched_3p_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco")
tau_matched_cuts_tpt_1p=signal_sample_file.Get("tau_matched_1p_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt")
tau_matched_cuts_tpt_3p=signal_sample_file.Get("tau_matched_3p_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt")

#tau_matched_cuts_1p=signal_sample_file.Get("tau_matched_cuts_1p")
#tau_matched_cuts_3p=signal_sample_file.Get("tau_matched_cuts_3p")
#tau_matched_cuts_tpt_1p=signal_sample_file.Get("tau_matched_cuts_tpt_1p")
#tau_matched_cuts_tpt_3p=signal_sample_file.Get("tau_matched_cuts_tpt_3p")


tau_matched_cuts_1p.SetDirectory(0)
tau_matched_cuts_3p.SetDirectory(0)
tau_matched_cuts_tpt_1p.SetDirectory(0)
tau_matched_cuts_tpt_3p.SetDirectory(0)

signal_sample_file.Close()

#1 PRONG INFO

tau_truth_cuts_1p=tau_matched_cuts_1p.GetBinContent(2)
tau_truth_cuts_1p_error=tau_matched_cuts_1p.GetBinError(2)
tau_fake_cuts_1p=tau_matched_cuts_1p.GetBinContent(1)
tau_fake_cuts_1p_error=tau_matched_cuts_1p.GetBinError(1)

tau_truth_cuts_tpt_1p=tau_matched_cuts_tpt_1p.GetBinContent(2)
tau_truth_cuts_tpt_1p_error=tau_matched_cuts_tpt_1p.GetBinError(2)
tau_fake_cuts_tpt_1p=tau_matched_cuts_tpt_1p.GetBinContent(1)
tau_fake_cuts_tpt_1p_error=tau_matched_cuts_tpt_1p.GetBinError(1)

#3 PRONG INFO

tau_truth_cuts_3p=tau_matched_cuts_3p.GetBinContent(2)
tau_truth_cuts_3p_error=tau_matched_cuts_3p.GetBinError(2)
tau_fake_cuts_3p=tau_matched_cuts_3p.GetBinContent(1)
tau_fake_cuts_3p_error=tau_matched_cuts_3p.GetBinError(1)

tau_truth_cuts_tpt_3p=tau_matched_cuts_tpt_3p.GetBinContent(2)
tau_truth_cuts_tpt_3p_error=tau_matched_cuts_tpt_3p.GetBinError(2)
tau_fake_cuts_tpt_3p=tau_matched_cuts_tpt_3p.GetBinContent(1)
tau_fake_cuts_tpt_3p_error=tau_matched_cuts_tpt_3p.GetBinError(1)

hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_Sherpa.root
hadd Target path: MC.root:/
hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_Sherpa.root
hadd Target path: MC.root:/
hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_Sherpa.root
hadd Target path: MC.root:/
hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_Sherpa.root
hadd Target path: MC.root:/


In [17]:
######### CALCULATION OF RQCD #############

for k in histos:
    rebin=histos[k][0]
    histo=k

    MC_CR_OS=ROOT.TFile.Open("CR_OS/MC.root","READ")
    Data_CR_OS = ROOT.TFile.Open("CR_OS/Data.root","READ")
    MC_CR_SS=ROOT.TFile.Open("CR_SS/MC.root","READ")
    Data_CR_SS = ROOT.TFile.Open("CR_SS/Data.root","READ")

    a=k

    mc_cr_os=MC_CR_OS.Get(histo)
    data_cr_os=Data_CR_OS.Get(histo)
    mc_cr_ss=MC_CR_SS.Get(histo)
    data_cr_ss=Data_CR_SS.Get(histo)

    mc_cr_os.SetDirectory(0)
    data_cr_os.SetDirectory(0)
    mc_cr_ss.SetDirectory(0)
    data_cr_ss.SetDirectory(0)

    MC_CR_OS.Close()
    Data_CR_OS.Close()
    MC_CR_SS.Close()
    Data_CR_SS.Close()

    data_cr_os.Add(mc_cr_os,-1)
    data_cr_ss.Add(mc_cr_ss,-1)
    
    # STORE STAT. UNCERTAINTIES FOR CONTROL REGION

    uncer_cr_os_1prong=ctypes.c_double()
    uncer_cr_ss_1prong=ctypes.c_double()
    uncer_cr_os_3prong=ctypes.c_double()
    uncer_cr_ss_3prong=ctypes.c_double()
    uncer_cr_os=ctypes.c_double()
    uncer_cr_ss=ctypes.c_double()
    
    sigma_rqcd=0

    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        RQCD=[data_cr_os.IntegralAndError(1,2,uncer_cr_os_1prong)/data_cr_ss.IntegralAndError(1,2,uncer_cr_ss_1prong),data_cr_os.IntegralAndError(3,4,uncer_cr_os_3prong)/data_cr_ss.IntegralAndError(3,4,uncer_cr_ss_3prong),data_cr_os.IntegralAndError(1,4,uncer_cr_os)/data_cr_ss.IntegralAndError(1,4,uncer_cr_ss)]
        
        integral_cr_os_1prong=data_cr_os.Integral(1,2)
        integral_cr_ss_1prong=data_cr_ss.Integral(1,2)
        integral_cr_os_3prong=data_cr_os.Integral(3,4)
        integral_cr_ss_3prong=data_cr_ss.Integral(3,4)
        integral_cr_os=data_cr_os.Integral()
        integral_cr_ss=data_cr_ss.Integral()
        
        sigma_rqcd=[RQCD[0]*((uncer_cr_os_1prong.value/integral_cr_os_1prong)**2+(uncer_cr_ss_1prong.value/integral_cr_ss_1prong)**2)**(0.5),RQCD[1]*((uncer_cr_os_3prong.value/integral_cr_os_3prong)**2+(uncer_cr_ss_3prong.value/integral_cr_ss_3prong)**2)**(0.5),RQCD[2]*((uncer_cr_os.value/integral_cr_os)**2+(uncer_cr_ss.value/integral_cr_ss)**2)**(0.5)]
        
        print(k)
        print("RQCD_1p=",round(RQCD[0],3))
        print("RQCD_3p=",round(RQCD[1],3))
        
        print("Sigma(RQCD_1p)=",round(sigma_rqcd[0],3))
        print("Sigma(RQCD_3p)=",round(sigma_rqcd[1],3))
        
    else :
        RQCD=data_cr_os.IntegralAndError(1,data_cr_os.GetNbinsX()+1,uncer_cr_os)/data_cr_ss.IntegralAndError(1,data_cr_ss.GetNbinsX()+1,uncer_cr_ss)
        
        integral_cr_os=data_cr_os.Integral(1,data_cr_os.GetNbinsX()+1)
        integral_cr_ss=data_cr_ss.Integral(1,data_cr_os.GetNbinsX()+1)
        
        sigma_rqcd=RQCD*((uncer_cr_os.value/integral_cr_os)**2+(uncer_cr_ss.value/integral_cr_ss)**2)**(0.5)

        
        print(k)
        print("RQCD=",round(RQCD,3))
        print("Sigma(RQCD)=",round(sigma_rqcd,3))
    
    # HISTOGRAM FOR SR_OS

    MC_SR_OS=ROOT.TFile.Open("SR_OS/MC.root","READ")
    Data_SR_OS = ROOT.TFile.Open("SR_OS/Data.root","READ")
    MC_SR_SS=ROOT.TFile.Open("SR_SS/MC.root","READ")
    Data_SR_SS = ROOT.TFile.Open("SR_SS/Data.root","READ")

    mc_sr_os=MC_SR_OS.Get(histo)
    data_sr_os=Data_SR_OS.Get(histo)
    mc_sr_ss=MC_SR_SS.Get(histo)
    data_sr_ss=Data_SR_SS.Get(histo)

    mc_sr_os.SetDirectory(0)
    data_sr_os.SetDirectory(0)
    mc_sr_ss.SetDirectory(0)
    data_sr_ss.SetDirectory(0)

    MC_SR_OS.Close()
    Data_SR_OS.Close()
    MC_SR_SS.Close()
    Data_SR_SS.Close()

    ob=data_sr_ss.GetNbinsX()
    s=data_sr_ss.GetXaxis().GetBinLowEdge(1)
    e=data_sr_ss.GetXaxis().GetBinUpEdge(data_sr_ss.GetNbinsX())
    n=5
    if rebin=="yy" or rebin=="yn":
        rebining=biner(s,histos[k][1],e,histos[k][2],histos[k][3])
        nb=len(rebining)-1
        mc_sr_ss=mc_sr_ss.Rebin(nb,"mc_sr_ss",rebining)
        data_sr_ss=data_sr_ss.Rebin(nb,"data_sr_ss",rebining)
        
        mc_sr_os=mc_sr_os.Rebin(nb,"mc_sr_os",rebining)
        data_sr_os=data_sr_os.Rebin(nb,"data_sr_os",rebining)

    MJ_BG=data_sr_ss.Clone()
    MJ_BG.Add(mc_sr_ss,-1)
    
    # SETTING BINS WITH NEGATIVE VALUES TO 0
    for j in range(1,MJ_BG.GetNbinsX()+2):
        if MJ_BG.GetBinContent(j)<0:
            MJ_BG.SetBinContent(j,0.0)
    
    # SCALING MJ_BG SHAPE BY RQCD
    
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        
        MJ_BG.SetBinContent(2,RQCD[0]*MJ_BG.GetBinContent(2))
        MJ_BG.SetBinContent(4,RQCD[1]*MJ_BG.GetBinContent(4))
        MJ_BG.SetBinError(2,RQCD[0]*MJ_BG.GetBinError(2))
        MJ_BG.SetBinError(4,RQCD[1]*MJ_BG.GetBinError(4))
        
    else : 
        X=MJ_BG.Integral(1,MJ_BG.GetNbinsX()+1)
        if X<0:
            X=0
            print("SR_SS Yield",X)
            MJ_BG.Scale(RQCD)
        else :
            print("SR_SS Yield",X)
            MJ_BG.Scale(RQCD)
    print("\n")
    
    # ADDING CONTROL REGION STAT. UNCERTAINTIES TO MJ BACKGROUND

    if k=="tau_nprongs_cuts" or k=="tau_nprongs_cuts_tpt":
        MJ_BG.SetBinError(2,(MJ_BG.GetBinError(2)**2+(MJ_BG.GetBinContent(2)**2)*(sigma_rqcd[0]/RQCD[0])**2)**(0.5))
        MJ_BG.SetBinError(4,(MJ_BG.GetBinError(2)**2+(MJ_BG.GetBinContent(4)**2)*(sigma_rqcd[1]/RQCD[1])**2)**(0.5))
    else :
        for j in range(1,MJ_BG.GetNbinsX()+2):
            MJ_BG.SetBinError(j,(MJ_BG.GetBinError(j)**2+(MJ_BG.GetBinContent(j)**2)*(sigma_rqcd/RQCD)**2)**(0.5))
            
    # SAVING MJ BG HISTOGRAM

    ##############################
    file=ROOT.TFile.Open("SR_OS/MJ.root","update")
    MJ_BG.Write(a)
    file.Close()
    ##############################

    #PLOTS

    ##### LOADING INFORMATION ABOUT EACH SAMPLE #####
    
    Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
    Signal=ROOT.TFile.Open("SR_OS/"+signal_sample,"READ")
    Ztt=ROOT.TFile.Open("SR_OS/"+ztt_sample,"READ")
    Higgs=ROOT.TFile.Open("SR_OS/Higgs.root","READ")
    VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
    Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
    Zjets=ROOT.TFile.Open("SR_OS/Zjets.root","READ")
    Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
    MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")
    Error=ROOT.TFile.Open("SR_OS/MC.root","READ")

    data=Data.Get(a)
    signal=Signal.Get(a)
    ztt=Ztt.Get(a)
    higgs=Higgs.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    mj=MJ.Get(a)
    error=Error.Get(a)

    signal.SetDirectory(0)
    ztt.SetDirectory(0)
    higgs.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    mj.SetDirectory(0)
    error.SetDirectory(0)

    Signal.Close()
    Ztt.Close()
    Higgs.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    MJ.Close()
    Error.Close()
    
    ####### DECLARING STACK HISTOGRAM ######

    hs = ROOT.THStack("hs","")
    
    ###### REBINING HISTOGRAMS #######
    
    if rebin=="yy" or rebin=="yn":
        signal=signal.Rebin(nb,"signal",rebining)
        ztt=ztt.Rebin(nb,"ztt",rebining)
        higgs=higgs.Rebin(nb,"higgs",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        error=error.Rebin(nb,"error",rebining)
        
    ##### SCALING W+JETS
    
    wjets_sf=0.8
    sigma_wjets_sf=0.2
    
    w.Scale(1/wjets_sf)
    
    for j in range(1,w.GetNbinsX()+1):
        w.SetBinError(j,(w.GetBinError(j)**2+(w.GetBinContent(j)**2)*(sigma_wjets_sf/wjets_sf)**2)**0.5)

    ###### CALCULATING THE RATIO MC/DATA ####### 
    
    ratio=w.Clone()
    lista = ROOT.TList()
    lista.Add(mj)
    lista.Add(z)
    lista.Add(vv)
    lista.Add(tt)
    lista.Add(t)
    lista.Add(higgs)
    lista.Add(ztt)
    lista.Add(signal)
    ratio.Merge(lista)
    ratio.Divide(data)
    ratio_error=ratio.Clone()
    
    ###### NORMALIZING HISTOGRAM USING VARIABLE BINING #######

    if rebin=="yy":
        hist_list=[signal,data,vv,tt,t,z,w,mj,error,ztt,higgs]
        normalization(hist_list,histos[k][2])
        
    ###### ADDING COLOURS TO THE HISTOGRAMS #######
    
    signal.SetFillColor(ROOT.kOrange+7)
    signal.SetLineWidth(0)
    ztt.SetLineColor(ROOT.kViolet)
    ztt.SetFillColor(ROOT.kViolet)
    higgs.SetFillColor(ROOT.kWhite)
    higgs.SetLineColor(ROOT.kWhite)
    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen+1)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed+1)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue+1)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow+1)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan+1)
    mj.SetLineColor(ROOT.kMagenta+1)
    mj.SetFillColor(ROOT.kMagenta)
    data.SetLineColor(ROOT.kBlack)
    error.SetFillColor(ROOT.kBlack)
    error.SetFillStyle(3151)
    ratio_error.SetFillColorAlpha(ROOT.kBlue, 0.35);
    
    
    ###### CREATING STACK HISTOGRAM #######
    
    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(higgs)
    hs.Add(mj)
    hs.Add(ztt)
    hs.Add(signal)
    
    error.Add(mj,1)
    
    ##### DRAWING CANVAS #######

    canvas = ROOT.TCanvas("canvas")
    canvas.cd()
    
    ##### DRAWING TOP PAD, SETTING MARGINS #######

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.01)
    pad1.SetRightMargin(0.01)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()
    
    ###### LOG SCALE OPTION FOR SOME HISTOGRAMS #######
    
    pad1.SetLogy(1)

    #### DRAWING HISTOGRAMS #####     
        
    data.SetMarkerStyle(ROOT.kFullCircle)
    data.SetMarkerSize(0.7)
    data.Draw("pe")
    error.Draw("E2 same")
    hs.Draw("HIST same")
    error.Draw("E2 same")
    data.Draw("pe same")
    
    if(not production):
    
    ##### DRAWING THE STATISTICS BOX #####
    
        gStyle.SetOptStat(1111111)
    
    ###### SETTING STATS BOX POSITION ######
    
        gStyle.SetStatY(0.97);                
        gStyle.SetStatX(0.20);
        gStyle.SetStatW(0.12);                
        gStyle.SetStatH(0.12);
        
    ###### ADDING A LABEL ACCORDING TO THE CHANNEL ######
    
        data.SetTitle("")
        l=ROOT.TLatex()
        l.SetNDC ()
        l.SetTextSize(0.125)
        l.DrawLatex(0.80,0.85,"Z#rightarrow#tau#tau#rightarrow#tau e")
        
    else :
        gStyle.SetOptStat(0)
        data.SetTitle("")
        l=ROOT.TLatex()
        l.SetNDC ()
        l.SetTextSize(0.062)
        l.DrawLatex(0.15,0.85,"Z#rightarrow#tau#tau#rightarrow#tau e")
    
    ###### DRAWING CUT LINES ######
    
    if len(histos[k])>8:
        cut1 = ROOT . TLine (histos[k][8] ,0 ,histos[k][8],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut1 . SetLineColor ( ROOT . kRed+1 )
        cut1 . SetLineWidth (2)
        cut1.SetLineStyle(2)
        
        cut2 = ROOT . TLine (histos[k][9] ,0 ,histos[k][9],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut2 . SetLineColor ( ROOT . kRed+1 )
        cut2 . SetLineWidth (2)
        cut2.SetLineStyle(2)
        
        cut1.Draw("same")
        cut2.Draw("same")
        
    data.GetXaxis().Draw()
    pad1.RedrawAxis()
    

    ##### ADJUSTING THE RANGE AND THE TITLE OF THE AXIS #####
    
    if len(histos[k])>7:
        e=histos[k][7]

    pad1.SetTicks(1,1)
    pad1.Update()
    data . GetYaxis (). SetRangeUser (1 ,6*histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (s ,e)
    data . GetYaxis (). SetTitle (histos[k][4])
    data . GetXaxis (). SetTitle (histos[k][5])
    data . GetYaxis (). SetTitleSize (0.07)
    data. GetYaxis (). SetTitleOffset (0.59)
    data.GetYaxis().SetLabelSize(0.055)
    data . GetXaxis (). SetTitleSize (0.01)

    ###### SETTING THE LEGENDS ON THE PLOT #####
    
    hs . GetXaxis (). SetRangeUser (s ,e)
    legend = ROOT . TLegend (0.45 ,0.80 ,0.95 ,0.95)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( ztt ," Ztautau")
    legend . AddEntry ( higgs ," Higgs")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jet ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend . AddEntry ( mj,"MJ")
    legend . AddEntry ( error,"MC Stat. Error")
    legend.SetTextAlign(22)
    legend.SetNColumns(3)
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (2)
    legend . Draw ()
    hs.SetTitle(a)
    
    ###### DRAWING PAD2 OF THE CANVAS, DRAWING THE RATIO MC/DATA ######

    canvas.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.00 ,1 ,0.35)
    pad2.SetRightMargin(0.01)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetMarkerStyle(ROOT.kFullCircle)
    ratio.SetMarkerSize(0.7)
    ratio.Draw ("p HIST")
    ratio_error.Draw ("E2 same")
    ratio.Draw ("p hist same")
    ratio.SetStats(0)
    ratio.SetStats(0)
    ratio.SetTitle("")
    ratio . GetYaxis (). SetRangeUser (0 ,2.0)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.12)
    ratio . GetYaxis (). SetTitleOffset (0.28)
    ratio . GetXaxis (). SetTitle (histos[k][5])
    ratio . GetXaxis (). SetTitleSize (0.14)
    ratio . GetXaxis (). SetTitleOffset (1.0)
    ratio.GetXaxis().SetLabelSize(0.14)
    ratio.GetYaxis().SetLabelSize(0.08)

    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line2 = ROOT . TLine (s ,1.2 ,e,1.2)
    line2 . SetLineColor ( ROOT . kBlack )
    line2 . SetLineWidth (1)
    line2 . SetLineStyle(2)
    line3 = ROOT . TLine (s ,0.8 ,e,0.8)
    line3 . SetLineColor ( ROOT . kBlack )
    line3 . SetLineWidth (1)
    line3 . SetLineStyle(2)
    line4 = ROOT . TLine (s ,1.4 ,e,1.4)
    line4 . SetLineColor ( ROOT . kBlack )
    line4 . SetLineWidth (1)
    line4 . SetLineStyle(2)
    line5 = ROOT . TLine (s ,0.6 ,e,0.6)
    line5 . SetLineColor ( ROOT . kBlack )
    line5 . SetLineWidth (1)
    line5 . SetLineStyle(2)
    line6 = ROOT . TLine (s ,1.6 ,e,1.6)
    line6 . SetLineColor ( ROOT . kBlack )
    line6 . SetLineWidth (1)
    line6 . SetLineStyle(2)
    line7 = ROOT . TLine (s ,0.4 ,e,0.4)
    line7 . SetLineColor ( ROOT . kBlack )
    line7 . SetLineWidth (1)
    line7 . SetLineStyle(2)
    line8 = ROOT . TLine (s ,1.8 ,e,1.8)
    line8 . SetLineColor ( ROOT . kBlack )
    line8 . SetLineWidth (1)
    line8 . SetLineStyle(2)
    line9 = ROOT . TLine (s ,0.2 ,e,0.2)
    line9 . SetLineColor ( ROOT . kBlack )
    line9 . SetLineWidth (1)
    line9 . SetLineStyle(2)
    line . Draw (" same ")
    line2 . Draw (" same ")
    line3 . Draw (" same ")
    line4 . Draw (" same ")
    line5 . Draw (" same ")
    line6 . Draw (" same ")
    line7 . Draw (" same ")
    line8 . Draw (" same ")
    line9 . Draw (" same ")
    line . Draw (" same ")
    
    ##### DRAWING THE CUT LINES IN CASE IT APPLIES #######
    
    if len(histos[k])>8:
        cut1.Draw("same")
        cut2.Draw("same")
    pad2.RedrawAxis()
    
    ###### SAVING THE HISTOGRAMS AS PDF FILES ########
    
    canvas.Print(a+".pdf")
    
    ####### PRINTING THE YIELDS FOR EVERY SAMPLE WITH THE STAT ERRORS #######
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        print("SF 1+3 prongs ----------------------------------------------")
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t,"mj":mj}
        dic_numbers={"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0],"mj":[0,0]}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(0,1000,error)
            dic_numbers[i][0]=a
            dic_numbers[i][1]=error.value
            
        if "tpt" in k:
            tau_dic={"signal_truth":[tau_truth_cuts_tpt_1p+tau_truth_cuts_tpt_3p,tau_truth_cuts_tpt_1p_error+tau_truth_cuts_tpt_3p_error],"signal_fake":[tau_fake_cuts_tpt_1p+tau_fake_cuts_tpt_3p,tau_fake_cuts_tpt_1p_error+tau_fake_cuts_tpt_3p_error]}
        else:
            tau_dic={"signal_truth":[tau_truth_cuts_1p+tau_truth_cuts_3p,tau_truth_cuts_1p_error+tau_truth_cuts_3p_error],"signal_fake":[tau_fake_cuts_1p+tau_fake_cuts_3p,tau_fake_cuts_1p_error+tau_fake_cuts_3p_error]}
        dic_numbers={**tau_dic,**dic_numbers,"RQCD":[RQCD[2],sigma_rqcd[1]]}
        
        dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
        display(dataframe.round(decimals=3))
        
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        print("SF 1 prong ----------------------------------------------")
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t,"mj":mj}
        dic_numbers={"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0],"mj":[0,0]}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(1,2,error)
            dic_numbers[i][0]=a
            dic_numbers[i][1]=error.value
        if "tpt" in k:
            tau_dic={"signal_truth":[tau_truth_cuts_tpt_1p,tau_truth_cuts_tpt_1p_error],"signal_fake":[tau_fake_cuts_tpt_1p,tau_fake_cuts_tpt_1p_error]}
        else:
            tau_dic={"signal_truth":[tau_truth_cuts_1p,tau_truth_cuts_1p_error],"signal_fake":[tau_fake_cuts_1p,tau_fake_cuts_1p_error]}
        dic_numbers={**tau_dic,**dic_numbers,"RQCD":[RQCD[0],sigma_rqcd[2]]}
            
        dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
        display(dataframe.round(decimals=3))
            
    if k=="tau_nprongs_cuts_tpt" or k=="tau_nprongs_cuts":
        print("SF 3 prong ----------------------------------------------")
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t,"mj":mj}
        dic_numbers={"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0],"mj":[0,0]}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(3,4,error)
            dic_numbers[i][0]=a
            dic_numbers[i][1]=error.value
        if "tpt" in k:
            tau_dic={"signal_truth":[tau_truth_cuts_tpt_3p,tau_truth_cuts_tpt_3p_error],"signal_fake":[tau_fake_cuts_tpt_3p,tau_fake_cuts_tpt_3p_error]}
        else:
            tau_dic={"signal_truth":[tau_truth_cuts_3p,tau_truth_cuts_3p_error],"signal_fake":[tau_fake_cuts_3p,tau_fake_cuts_3p_error]}
        dic_numbers={**tau_dic,**dic_numbers,"RQCD":[RQCD[1],sigma_rqcd[1]]}
        
        dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
        display(dataframe.round(decimals=3))
        print('\n') 

delta_phi
RQCD= 1.073
Sigma(RQCD)= 0.337
SR_SS Yield 88.41215801239014


n_bjets
RQCD= 2.087
Sigma(RQCD)= 0.995
SR_SS Yield 47.66678428649902


lepiso
RQCD= 1.505
Sigma(RQCD)= 0.733
SR_SS Yield 53.364808082580566


n_jets_interval
RQCD= 1.755
Sigma(RQCD)= 0.62
SR_SS Yield 68.7883529663086


lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt
RQCD= 2.059
Sigma(RQCD)= 0.979
SR_SS Yield 49.576999843120575


tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt
RQCD= 2.059
Sigma(RQCD)= 0.979
SR_SS Yield 48.842074394226074


delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt
RQCD= 2.059
Sigma(RQCD)= 0.979
SR_SS Yield 47.598976612091064


delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt
RQCD= 2.059
Sigma(RQCD)= 0.979
SR_SS Yield 47.42211329936981


delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_p

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file delta_phi.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file n_bjets.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file lepiso.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file n_jets_interval.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mlt_ebdt_mreco_tpt.pdf has be

In [23]:
import os
import shutil
if 'OnlySignal' in os.listdir():
    shutil.rmtree('OnlySignal')
os.mkdir('OnlySignal')

for k in histos:
    rebin=histos[k][0]
    histo=k

    MC_CR_OS=ROOT.TFile.Open("CR_OS/MC.root","READ")
    Data_CR_OS = ROOT.TFile.Open("CR_OS/Data.root","READ")
    MC_CR_SS=ROOT.TFile.Open("CR_SS/MC.root","READ")
    Data_CR_SS = ROOT.TFile.Open("CR_SS/Data.root","READ")

    a=k

    mc_cr_os=MC_CR_OS.Get(histo)
    data_cr_os=Data_CR_OS.Get(histo)
    mc_cr_ss=MC_CR_SS.Get(histo)
    data_cr_ss=Data_CR_SS.Get(histo)

    mc_cr_os.SetDirectory(0)
    data_cr_os.SetDirectory(0)
    mc_cr_ss.SetDirectory(0)
    data_cr_ss.SetDirectory(0)

    MC_CR_OS.Close()
    Data_CR_OS.Close()
    MC_CR_SS.Close()
    Data_CR_SS.Close()

    data_cr_os.Add(mc_cr_os,-1)
    data_cr_ss.Add(mc_cr_ss,-1)

    RQCD=data_cr_os.Integral()/data_cr_ss.Integral()

    # HISTOGRAM FOR SR_OS

    MC_SR_OS=ROOT.TFile.Open("SR_OS/MC.root","READ")
    Data_SR_OS = ROOT.TFile.Open("SR_OS/Data.root","READ")
    MC_SR_SS=ROOT.TFile.Open("SR_SS/MC.root","READ")
    Data_SR_SS = ROOT.TFile.Open("SR_SS/Data.root","READ")

    mc_sr_os=MC_SR_OS.Get(histo)
    data_sr_os=Data_SR_OS.Get(histo)
    mc_sr_ss=MC_SR_SS.Get(histo)
    data_sr_ss=Data_SR_SS.Get(histo)

    mc_sr_os.SetDirectory(0)
    data_sr_os.SetDirectory(0)
    mc_sr_ss.SetDirectory(0)
    data_sr_ss.SetDirectory(0)

    MC_SR_OS.Close()
    Data_SR_OS.Close()
    MC_SR_SS.Close()
    Data_SR_SS.Close()

    ob=data_sr_ss.GetNbinsX()
    s=data_sr_ss.GetXaxis().GetBinLowEdge(1)
    e=data_sr_ss.GetXaxis().GetBinUpEdge(data_sr_ss.GetNbinsX())
    n=5
    if rebin=="yy" or rebin=="yn":
        rebining=biner(s,histos[k][1],e,histos[k][2],histos[k][3])
        nb=len(rebining)-1
        mc_sr_ss=mc_sr_ss.Rebin(nb,"mc_sr_ss",rebining)
        data_sr_ss=data_sr_ss.Rebin(nb,"data_sr_ss",rebining)


    MJ_BG=data_sr_ss.Clone()
    MJ_BG.Add(mc_sr_ss,-1)
    MJ_BG.Scale(RQCD)

    # SAVING MJ BG HISTOGRAM

    ##############################
    file=ROOT.TFile.Open("SR_OS/MJ.root","update")
    MJ_BG.Write(a)
    file.Close()
    ##############################

    #PLOTS

    
    ##### LOADING INFORMATION ABOUT EACH SAMPLE #####
    
    Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
    Signal=ROOT.TFile.Open("SR_OS/Signal.root","READ")
    VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
    Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
    Zjets=ROOT.TFile.Open("SR_OS/Zjets.root","READ")
    Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
    MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")
    Error=ROOT.TFile.Open("SR_OS/Signal.root","READ")

    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    mj=MJ.Get(a)
    error=Error.Get(a)

    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    mj.SetDirectory(0)
    error.SetDirectory(0)

    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    MJ.Close()
    Error.Close()
    
    ##### REBINING HISTOGRAMS #####

    if rebin=="yy" or rebin=="yn":
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        error=error.Rebin(nb,"error",rebining)
    
    ##### IF HISTOGRAMS HAVE VARIABLE BINNING, NORMALIZE IT #####
    
    if rebin=="yy":
        hist_list=[signal,data,vv,tt,t,z,w,mj,error]
        normalization(hist_list)
        
    
    ##### SUBSTRACTING EW BG FROM DATA #####
    
    data_minus_BG=data.Clone()
    data_minus_BG.Add(w,-1)
    data_minus_BG.Add(z,-1)
    data_minus_BG.Add(vv,-1)
    data_minus_BG.Add(tt,-1)
    data_minus_BG.Add(t,-1)
    data_minus_BG.Add(mj,-1)
    
    ##### DEFINING RATIO SIGNAL VS DATA-BG #####
    
    ratio=signal.Clone()
    ratio.Divide(data_minus_BG)
    ratio_error=ratio.Clone()
    
    ##### COLOURING THE HISTOGRAMS ##### 
    
    signal.SetFillColor(ROOT.kAzure+8)
    data_minus_BG.SetLineColor(ROOT.kBlack)
    error.SetFillColorAlpha(ROOT.kBlue,0.5)
    ratio_error.SetFillColorAlpha(ROOT.kBlue,0.5)
    
    ##### DRAWING THE STATISTICS BOX #####
    gStyle.SetOptStat(1111111)

    
    ##### DRAWING CANVAS #######
    canvas = ROOT.TCanvas("canvas")
    canvas.cd()
    
    ##### DRAWING TOP PAD, SETTING MARGINS #######

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.03)
    pad1.SetRightMargin(0.03)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()
    
    ##### SETTING STATS BOX POSITION #######

    gStyle.SetStatY(0.93);                
    gStyle.SetStatX(0.95);
    gStyle.SetStatW(0.1);                
    gStyle.SetStatH(0.1);
    
    #### OPTION FOR LOG PLOT #####
    
    #if "rnn_score" in k:
        #pad1.SetLogy(1)

    #### DRAWING HISTOGRAMS #####    
    
    data_minus_BG.SetMarkerStyle(ROOT.kFullCircle)
    data_minus_BG.SetMarkerSize(0.7)
    data_minus_BG.Draw("pe")
    signal.Draw("HIST same")
    error.Draw("E2 same")
    data_minus_BG.Draw("pe same")
    
    
    #### DRAWING CUT LINES #####  
    
    if len(histos[k])>8:
        cut1 = ROOT . TLine (histos[k][8] ,0 ,histos[k][8],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut1 . SetLineColor ( ROOT . kRed+1 )
        cut1 . SetLineWidth (2)
        cut1.SetLineStyle(2)
        
        cut2 = ROOT . TLine (histos[k][9] ,0 ,histos[k][9],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut2 . SetLineColor ( ROOT . kRed+1 )
        cut2 . SetLineWidth (2)
        cut2.SetLineStyle(2)
        
        cut1.Draw("same")
        cut2.Draw("same")
    
    data_minus_BG.GetXaxis().Draw()
    pad1.RedrawAxis()

    ###### ADDING A LABEL ACCORDING TO THE CHANNEL ######
    
    data_minus_BG.SetTitle("")
    l=ROOT.TLatex()
    l.SetNDC ()
    l.DrawLatex(0.6,0.88,"Z#rightarrow#tau#tau#rightarrow#tau#mu")
    
    ##### ADJUSTING THE RANGE AND THE TITLE OF THE AXIS #####
    
    if len(histos[k])>7:
        e=histos[k][7]

    pad1.SetTicks(1,1)
    pad1.Update()
    data_minus_BG.GetYaxis (). SetRangeUser (1 ,histos[k][6]*data_minus_BG.GetBinContent(data_minus_BG.GetMaximumBin()))
    data_minus_BG.GetXaxis (). SetRangeUser (s ,e)
    data_minus_BG.GetYaxis (). SetTitle (histos[k][4])
    data_minus_BG.GetXaxis (). SetTitle (histos[k][5])

    ###### SETTING THE LEGENDS ON THE PLOT #####
    
    legend = ROOT . TLegend (0.7 ,0.73 ,0.8 ,0.93)
    legend . AddEntry ( data_minus_BG ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( error,"MC Stat. Error")
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (2)
    legend . Draw ()
    
    ###### DRAWING PAD2 OF THE CANVAS, DRAWING THE RATIO MC/DATA ######

    canvas.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.00 ,1 ,0.35)
    pad2.SetRightMargin(0.03)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.2)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetMarkerStyle(ROOT.kFullCircle)
    ratio.SetMarkerSize(0.7)
    ratio.Draw ("p HIST")
    ratio_error.Draw ("E2 same")
    ratio.Draw ("p hist same")
    ratio.SetStats(0)
    ratio.SetStats(0)
    ratio.SetTitle("")
    ratio . GetYaxis (). SetRangeUser (0 ,2.0)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.10)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    ratio . GetXaxis (). SetTitle (histos[k][5])
    ratio . GetXaxis (). SetTitleSize (0.09)
    ratio.GetXaxis().SetLabelSize(0.10)
    ratio.GetYaxis().SetLabelSize(0.08)
    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line2 = ROOT . TLine (s ,1.2 ,e,1.2)
    line2 . SetLineColor ( ROOT . kBlack )
    line2 . SetLineWidth (1)
    line2 . SetLineStyle(2)
    line3 = ROOT . TLine (s ,0.8 ,e,0.8)
    line3 . SetLineColor ( ROOT . kBlack )
    line3 . SetLineWidth (1)
    line3 . SetLineStyle(2)
    line4 = ROOT . TLine (s ,1.4 ,e,1.4)
    line4 . SetLineColor ( ROOT . kBlack )
    line4 . SetLineWidth (1)
    line4 . SetLineStyle(2)
    line5 = ROOT . TLine (s ,0.6 ,e,0.6)
    line5 . SetLineColor ( ROOT . kBlack )
    line5 . SetLineWidth (1)
    line5 . SetLineStyle(2)
    line6 = ROOT . TLine (s ,1.6 ,e,1.6)
    line6 . SetLineColor ( ROOT . kBlack )
    line6 . SetLineWidth (1)
    line6 . SetLineStyle(2)
    line7 = ROOT . TLine (s ,0.4 ,e,0.4)
    line7 . SetLineColor ( ROOT . kBlack )
    line7 . SetLineWidth (1)
    line7 . SetLineStyle(2)
    line8 = ROOT . TLine (s ,1.8 ,e,1.8)
    line8 . SetLineColor ( ROOT . kBlack )
    line8 . SetLineWidth (1)
    line8 . SetLineStyle(2)
    line9 = ROOT . TLine (s ,0.2 ,e,0.2)
    line9 . SetLineColor ( ROOT . kBlack )
    line9 . SetLineWidth (1)
    line9 . SetLineStyle(2)
    line . Draw (" same ")
    line2 . Draw (" same ")
    line3 . Draw (" same ")
    line4 . Draw (" same ")
    line5 . Draw (" same ")
    line6 . Draw (" same ")
    line7 . Draw (" same ")
    line8 . Draw (" same ")
    line9 . Draw (" same ")
    line . Draw (" same ")
    
    ##### DRAWING THE CUT LINES IN CASE IT APPLIES #######
    
    if len(histos[k])>8:
        cut1.Draw("same")
        cut2.Draw("same")
    pad2.RedrawAxis()
    
    ###### SAVING THE HISTOGRAMS AS PDF FILES ########
    
    os.chdir('OnlySignal/')
    canvas.Print(a+".pdf")
    os.chdir('../')

ReferenceError: attempt to access a null-pointer

Error in <TFile::TFile>: file SR_OS/Signal.root does not exist
Error in <TFile::TFile>: file SR_OS/Signal.root does not exist
